<a href="https://colab.research.google.com/github/lavanaythakral/BERT-Q-A/blob/master/BERT%20for%20QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Thesis project/Thesis project/datasets/WOF_split_into_sentences.csv')
sentences = list(df['Sentences'])
len(sentences)

3085

In [3]:
!pip install tensorflow==1.14

     |████████████████████████████████| 109.2MB 94kB/s 
     |████████████████████████████████| 491kB 45.6MB/s 
     |████████████████████████████████| 3.2MB 38.0MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


#Context Extraction 
Using TF-IDF cosine similarity

In [4]:
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk import wordpunct_tokenize, WordNetLemmatizer, sent_tokenize, pos_tag
from nltk.corpus import stopwords as sw, wordnet as wn
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
import string 
import json

In [5]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(sentences)
X = normalize(X)

In [6]:
X.shape

(3085, 6929)

In [7]:
def get_context(query,X,data_sentences):
  Question = vectorizer.transform([query])
  Question = normalize(Question)
  cosineSimilarities = cosine_similarity(Question, X).flatten()
  idx = cosineSimilarities.argsort()[::-1][:20]
  temp = ""
  print(query)
  for i in idx:
    if(cosineSimilarities[i] != 0):
      # if(i-1 >=0):
      #   temp = temp + data_sentences[i-1]
      temp = temp + data_sentences[i]
      # if(i+1 < len(data_sentences)):
      #   temp = temp + data_sentences[i+1]
  return temp

def make_json_list(queries,X,sentences):
  jsondata = {}
  data1 = {}  
  paras =[]
  gpt2_feed = []
  for idx,query in enumerate(queries):
    context = get_context(query,X,sentences)
    if(context == ""):
      gpt2_feed.append(query)
      continue
    qa1 = {}
    qa1["question"] = query
    qa1["id"] = str(idx)
    qa1["is_impossible"] = ""
    qas = [qa1]
    paragraph1 = {}
    paragraph1["qas"] =[qa1]
    paragraph1["context"] = context
    paras.append(paragraph1)

  data1["paragraphs"] = paras
  data1["title"] = "WOF"
  data = [data1]
  jsondata["version"] = "v2.0"
  jsondata["data"] = data
  jstr = json.dumps(jsondata,indent = 5)
  print(jstr)
  return jstr


def make_json(query,X,sentences,idx):
  context = get_context(query,X,sentences)
  if(context == ""):
    return 0
  jsondata = {}
  data1 = {}  
  qa1 = {}
  paragraph1 = {}
  qa1["question"] = query
  qa1["id"] = str(idx)
  qa1["is_impossible"] = ""
  qas = [qa1]
  paragraph1["qas"] =[qa1]
  paragraph1["context"] = context
  
  
  paras = [paragraph1]
  data1["paragraphs"] = paras
  data1["title"] = "WOF"
  data = [data1]
  jsondata["version"] = "v2.0"
  jsondata["data"] = data
  jstr = json.dumps(jsondata,indent = 5)
  print(json.dumps(jsondata,indent = 5))
  return jstr

In [8]:
test_questions = ["What is your name?","What is SLV?", "When is your birthday?" , "Who is Narayan?", "Who is your sister?", "where are you from?", "Where is Pentagon?" , "where did Trishul take off from?", "What is Prithvi?" ,"What is Astra?", "Did you have childhood friends?","Which school did you go to as a child?" , "when did you start liking Science?", "What was your role in Nike-Apache?" , "What are sounding rockets used for?" , "Are you religious?" , "What is SAM?" , "Who is Indira Gandhi?" , "Who conferred the Honorary Doctor of Science degree on to you?" , "What is TCV?"]
test_questions_B = ["How can I overcome difficulty?","Why did I fail?","How can I become successful?","How can I be more creative?","What is love?","Do you need to fail to succeed?","Is there an afterlife?","What is the meaning of life?","How do you judge a person?","What is the most important thing in you life?","What is your definition of success?","What type of person do I want to be?","How can you love yourself more today?","How can you change someone's life for the better today?","What inspire you the most?","What should I do next?","What should I have for dinner?","cheer up","what is your mission?"]
test_questions_C = ["Why do we create Art?", "What is free will?", "What is freedom","What is truth?","What is the soul?","What do you fear most?","Is there such a thing as luck","Are clowns funny or scary?","What are rights?","What is mercy?","What is self-esteem?","what is age?","Can money buy happiness?","Do you like music?","What are dreams?","How to be happy?","What is beauty?","Should you forgive people?","Is our universe real?","Does god exist?","What is friendship?","What is patriotism?","Are wars necessary?","What is time?","What do you think about time travel?","What is philosophy?","Who are you?","Do animals have feelings?","What is the goal of life?","Is lying okay?","Is humanity headed in the right direction?","What is wisdom?","What is waste of human potential?","What is funny?","Are animals moral?","is privacy a right?","What is your definition of evil?","Is censorship necessary?","Who are you?","How to be confident?","Is everyone's life value equal?"]

In [9]:
test_questions_philosophy = test_questions_B + test_questions_C

In [10]:
len(test_questions_philosophy)

60

In [ ]:
jstr = make_json_list(test_questions_C,X,sentences)

In [15]:
filename = "/content/drive/My Drive/Thesis project/Thesis project/datasets/jsons/context.json"
save_json(jstr,filename)

In [14]:
def save_json(jstr,filename):
  with open(filename, "w") as outfile: 
    outfile.write(jstr) 

#BERT Q/A

In [16]:
!git clone https://github.com/google-research/bert.git
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip

Cloning into 'bert'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 317.85 KiB | 4.07 MiB/s, done.
Resolving deltas: 100% (185/185), done.
--2020-10-01 18:45:44--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.124.128, 172.217.212.128, 172.217.214.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.124.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M  84.1MB/s    in 5.9s    

2020-10-01 18:45:50 (65.6 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  infl

In [17]:
cd bert

/content/bert


In [18]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is => ', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

TPU address is =>  grpc://10.33.99.162:8470


/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 12367101994050641944),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 6361398146712378670),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 7922992186653088150),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 14664204367298079330),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 2407478756424986262),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 13524195

In [19]:
BUCKET = 'bucket-bert-qa' #@param {type:"string"}
assert BUCKET, '*** Must specify an existing GCS bucket name ***'
output_dir_name = 'bert_output' #@param {type:"string"}
BUCKET_NAME = 'gs://{}'.format(BUCKET)
OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET,output_dir_name)
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: gs://bucket-bert-qa/bert_output *****


In [20]:
!cp "/content/drive/My Drive/Thesis project/Thesis project/datasets/jsons/context.json" "/content/bert"

In [63]:
!ls

context.json		    predicting_movie_reviews_with_bert_on_tf_hub.ipynb
CONTRIBUTING.md		    README.md
create_pretraining_data.py  requirements.txt
extract_features.py	    run_classifier.py
__init__.py		    run_classifier_with_tfhub.py
LICENSE			    run_pretraining.py
modeling.py		    run_squad.py
modeling_test.py	    sample_text.txt
multilingual.md		    tokenization.py
optimization.py		    tokenization_test.py
optimization_test.py


In [21]:
def run():
  !python run_squad.py \
    --vocab_file=$BUCKET_NAME/uncased_L-12_H-768_A-12/vocab.txt \
    --bert_config_file=$BUCKET_NAME/uncased_L-12_H-768_A-12/bert_config.json \
    --init_checkpoint=$OUTPUT_DIR/model.ckpt-10859 \
    --do_train=False \
    --max_query_length=30  \
    --do_predict=True \
    --predict_file=context.json \
    --predict_batch_size=8 \
    --n_best_size=3 \
    --max_seq_length=512 \
    --doc_stride=128 \
    --version_2_with_negative=True \
    --output_dir=output/

In [ ]:
run()

In [23]:
def load_json(filename):
  with open(filename) as f:
    res = json.load(f)
    return res

In [24]:
def process_individual(query,X,sentences,idx):
  jstr = make_json(query,X,sentences,idx)
  if(jstr == 0):
    print("No context")
    return 0 
  else:
    filename = "/content/bert/context.json"
    save_json(jstr,filename)
    run()
  return jstr

In [25]:
def make_prediction(questions,X,sentences):
  predictions = {}
  for idx,query in enumerate(questions):
    jstr = process_individual(query,X,sentences,idx)
    if(jstr == 0):
      continue
    filename = "/content/bert/output/predictions.json"
    res = load_json(filename)
    if(str(idx) in res):
      predictions[str(idx)] = res[str(idx)]
  return predictions

In [ ]:
pred = make_prediction(test_questions_C,X,sentences)

In [27]:
filename = "/content/drive/My Drive/Thesis project/Thesis project/datasets/post BERT/predictions_B.json"
p1 = json.dumps(pred)
save_json(p1,filename)

In [28]:
pred

{'0': '',
 '1': '',
 '10': 'I too could aspire to become whatever I wished',
 '11': 'sixty years of age',
 '12': 'The entire room was irradiated with happiness.',
 '13': "I filled my room with the music of Bismillah Khan's shehnai",
 '14': "two plans have evolved out of the nation's dreams",
 '15': 'I am happy to see your work',
 '16': '',
 '18': 'This is made real',
 '19': 'They exist at various levels of the human personality',
 '2': '',
 '20': 'equal warmth',
 '22': 'wars were waged over religious and ideological beliefs',
 '23': 'a second re-organization',
 '24': '',
 '25': 'one-to- one substitution',
 '27': 'I experienced mixed feelings',
 '28': 'the goal that makes the difference',
 '3': 'there is a great deal that most of us can individually do to increase our freedom',
 '30': '',
 '32': '',
 '34': 'moral compulsion',
 '35': '',
 '37': '',
 '39': 'tense',
 '4': 'Nandi',
 '40': 'His friendship embraced with equal warmth',
 '6': 'If there is one thing outsiders dislike, it is unpl

#Analysis

In [ ]:
!cp "/content/bert/output/predictions.json" "/content/drive/My Drive/Thesis project/Thesis project/datasets/post BERT"

In [ ]:
import csv
with open("/content/drive/My Drive/Thesis project/Thesis project/datasets/post BERT/questions_philosophy.csv","w") as f:
  writer = csv.writer(f)
  writer.writerow(["Questions"])
  for t in test_questions_B:
    writer.writerow([t]) 